In [ ]:
# 종목 이름을 입력하면 종목에 해당하는 코드를 불러와 # 네이버 금융(http://finance.naver.com)에 넣어줌 
def get_url(item_name, code_df): 
    code = code_df.query("name=='{}'".format(item_name))['code'].to_string(index=False).lstrip()
    url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code) 
#     url = 'http://finance.naver.com/item/sise.nhn?code={code}'.format(code=code) 
    print("요청 URL = {}".format(url)) 
    return url 


# 신라젠의 일자데이터 url 가져오기 
# item_name='신라젠' 
item_name='GS' 
url = get_url(item_name, code_df) 

# 일자 데이터를 담을 df라는 DataFrame 정의 
df = pd.DataFrame() 

# 1페이지에서 20페이지의 데이터만 가져오기 
for page in range(1, 21): 
    pg_url = '{url}&page={page}'.format(url=url, page=page) 
    df = df.append(pd.read_html(pg_url, header=0)[0], ignore_index=True) 
    # df.dropna()를 이용해 결측값 있는 행 제거 
    df = df.dropna() 
    # 상위 5개 데이터 확인하기 
    df.head()


# 한글로 된 컬럼명을 영어로 바꿔줌 
df_eng = df.rename(columns= {'날짜': 'date', '종가': 'close', '전일비': 'diff', '시가': 'open', '고가': 'high', '저가': 'low', '거래량': 'volume'}) 

df_eng['date'] = pd.to_datetime(df_eng['date'])
df_input = df_eng.set_index('date')

In [ ]:
def calculate_EGR (ds_input,
                     idx_datetime,
                     unit_period='Week',
                     return_span_Y_fitted=False):
  ds_sorted_input_with_date_time_index = ds_input.loc[idx_datetime].sort_index()
  idx_datetime_sorted = ds_sorted_input_with_date_time_index.index
  start_datetime = ds_sorted_input_with_date_time_index.index[0]
  end_datetime = ds_sorted_input_with_date_time_index.index[-1]
  inferred_freq = pd.infer_freq(ds_sorted_input_with_date_time_index.index)
  idx_datetime_range_span = pd.date_range(start = start_datetime, end = end_datetime, freq = inferred_freq)

  query_ts = ds_sorted_input_with_date_time_index.values.reshape(-1, 1)

  diff_datetime_sec = (end_datetime - start_datetime).total_seconds()

  from sklearn.linear_model import LinearRegression

  list_available_datetime_idx = [datetime_idx for datetime_idx, datetime in enumerate(idx_datetime_range_span) if datetime in list(idx_datetime_sorted)]
  list_span_datetime_idx = [datetime_idx for datetime_idx, _ in enumerate(idx_datetime_range_span)]


  # print("idx_datetime_range_span \n", idx_datetime_range_span)
  # print("list_available_datetime_idx \n", list_available_datetime_idx)
  # print("list_span_datetime_idx  \n", list_span_datetime_idx)

  # X_for_fitting = np.array(range(1, len(query_ts)+1)))
  X_for_fitting = np.array(list_available_datetime_idx)
  Y_for_fitting = query_ts

  line_fitter = LinearRegression()
  line_fitter.fit(X_for_fitting.reshape(-1, 1), Y_for_fitting)

  Y_fitted = line_fitter.predict(X_for_fitting.reshape(-1, 1)).reshape(1, -1)[0]

  X_after_filling = np.array(list_span_datetime_idx)
  Y_fitted_after_filling = line_fitter.predict(X_after_filling.reshape(-1, 1)).reshape(1, -1)[0]


  ds_Y_fitted_without_filling = pd.Series(index = idx_datetime_sorted, data = Y_fitted)
  ds_Y_fitted_after_filling = pd.Series(index = idx_datetime_range_span, data = Y_fitted_after_filling)

  if return_span_Y_fitted == True:
    ds_Y_fitted = ds_Y_fitted_after_filling
  else:
    ds_Y_fitted = ds_Y_fitted_without_filling 


  mean_during_window = ds_input.mean()

  effective_total_diff = ((line_fitter.coef_[0][0] * len(idx_datetime_range_span)) / mean_during_window)
  # print(mean_during_window)

  if mean_during_window == 0.0:
    effective_growth_rate = 0.0
  else:
    if unit_period == 'Hour':
      effective_growth_rate = (3600)*effective_total_diff / diff_datetime_sec
    elif unit_period == 'Day':
      effective_growth_rate = (24*3600)*effective_total_diff / diff_datetime_sec
    elif unit_period == 'Week':
      effective_growth_rate = (7*24*3600)*effective_total_diff / diff_datetime_sec
    elif unit_period == 'Month':
      effective_growth_rate =  (30*24*3600)*effective_total_diff / diff_datetime_sec
    else:
      assert unit_period in ['Hour', 'Day', 'Week', 'Month']

  # if mean_during_window == 0.0:
  #   return 0.0, 0.0, ds_Y_fitted
  # else:
  #   if unit_period == 'Hour':
  #     return effective_total_diff, (3600)*effective_total_diff / diff_datetime_sec, ds_Y_fitted
  #   elif unit_period == 'Day':
  #     return effective_total_diff, (24*3600)*effective_total_diff / diff_datetime_sec, ds_Y_fitted
  #   elif unit_period == 'Week':
  #     return effective_total_diff, (7*24*3600)*effective_total_diff / diff_datetime_sec, ds_Y_fitted
  #   elif unit_period == 'Month':
  #     return effective_total_diff, (30*24*3600)*effective_total_diff / diff_datetime_sec, ds_Y_fitted    
  #   else:
  #     assert unit_period in ['Hour', 'Day', 'Week', 'Month']

  return effective_total_diff, effective_growth_rate, ds_Y_fitted


ds_input = df_input['close']
idx_datetime = ds_input.index

effective_total_diff, effective_growth_rate, ds_Y_fitted = calculate_EGR(ds_input, idx_datetime, unit_period = 'Month')

In [ ]:
from dtw import *
from sklearn.preprocessing import MinMaxScaler
import numpy as np

dict_reference_trend_shape_input = {
    'Increasing': np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
    'Decreasing': np.array([9, 8, 7, 6, 5, 4, 3, 2, 1, 0])
}


def scale_reference_shape(dict_reference_trend_shape):

  list_trend_type = dict_reference_trend_shape.keys()

  dict_scaled_reference_trend_shape = {}

  for trend_type in list_trend_type:
    minMaxScaler = MinMaxScaler(feature_range=(-0.5, 0.5))
    reference_ts = dict_reference_trend_shape[trend_type].reshape(-1, 1)
    minMaxScaler.fit(reference_ts)
    dict_scaled_reference_trend_shape[trend_type] = minMaxScaler.transform(reference_ts)
    # dict_reference_trend_shape[trend_type]
    # dict_scaled_reference_trend_shape[trend_type] = 

  return dict_scaled_reference_trend_shape


def classify_shape(ds_input,
                   idx_datetime,
                   dict_reference_trend_shape,
                   after_sorting=True,
                   using_normalized_distance=True):
  if after_sorting == True:
    query_ts = ds_input.loc[idx_datetime].sort_index().values.reshape(-1, 1)
  else:
    query_ts = ds_input.loc[idx_datetime].values.reshape(-1, 1)

  from scipy.spatial.distance import euclidean

  # from sklearn.preprocessing import MinMaxScaler

  minMaxScaler = MinMaxScaler(feature_range=(-0.5, 0.5))
  minMaxScaler.fit(query_ts)
  query_ts_scaled = minMaxScaler.transform(query_ts)

  if np.var(query_ts_scaled) == 0:
    query_ts_scaled[:] = 0.0

  list_trend_type = dict_reference_trend_shape.keys()

  index_df_distance_result = pd.Index(list_trend_type, name = 'trend type')

  df_distance_result = pd.DataFrame(index = index_df_distance_result, columns = ['distance'])

  for trend_type in list_trend_type:

    reference_ts = dict_reference_trend_shape[trend_type]

    tmp_alignment = dtw(query_ts_scaled, reference_ts, keep_internals=True)
    
    if using_normalized_distance == True:
      df_distance_result.loc[trend_type]['distance'] = tmp_alignment.distance/len(query_ts)
    else:
      df_distance_result.loc[trend_type]['distance'] = tmp_alignment.distance

    df_distance_result = df_distance_result.astype('float')

  return df_distance_result['distance'].idxmin(), df_distance_result

  

dict_reference_trend_shape = scale_reference_shape(dict_reference_trend_shape_input)
shape_result, df_out = classify_shape(ds_input,
                   idx_datetime,
                   dict_reference_trend_shape,
                   after_sorting=True,
                   using_normalized_distance=True)